In [ ]:
# import os
# os.environ["NUMBA_DISABLE_JIT"] = "1"

from asian_option import AsianArithmetic, Settings, plot_solution, plot_difference_arithmetic

import numpy as np
from matplotlib import pyplot
from ipywidgets import IntProgress
from tqdm import tqdm

from open_atmos_jupyter_utils import show_plot, show_anim
from monte_carlo import BSModel, FixedStrikeGeometricAsianOption, FixedStrikeArithmeticAsianOption
import time

In [ ]:
def run_numeric_and_mc(params, nx=31, ny=41, nt=300, variant='call'):
    s_max = params.get('S_max', params['spot']*2)
    s_min = params.get('S_min', params['spot']/2)
    settings = Settings(T=params['T'], K=params['K'], r=params['r'], sgma=params['sgma'], S_max=s_max, S_min=s_min)
    simulation = AsianArithmetic(settings, nx=nx, ny=ny, nt=nt, variant=variant)
    
    
    history = []
    progbar = IntProgress(max = simulation.nt)
    display(progbar)
    start = time.time()
    for progbar.value in range(simulation.nt + 1):
        if progbar.value != 0:
            res = simulation.step()
        history.append(simulation.solver.advectee.get().copy())
    end = time.time()
    print(f"MPDATA elapsed time: {end - start}")
    print(f"Numeric price: {np.interp(params['spot'], simulation.S, history[-1][:,0])}")
    start = time.time()
    arithmetic_by_mc = np.zeros_like(simulation.S)
    for idx, spot in tqdm(enumerate(simulation.S)):
        model = BSModel(spot, settings.params.r, settings.params.sgma, settings.params.T, 1000)
        arithmetic_option = FixedStrikeArithmeticAsianOption(settings.params.T, settings.params.K, variant, model,  100000)
        arithmetic_by_mc[idx] = arithmetic_option.price_by_mc()
    end = time.time()
    print(f"MC elapsed time: {end - start}")
    _, ax = pyplot.subplots(1, 1, figsize=(10, 5))
    plot_solution(settings=settings,
                  frame_index=-1,
                  ax=ax,
                  history=history,
                  arithmetic_by_mc=arithmetic_by_mc,
                  S_linspace=simulation.S,
                  option_type='arithmetic call',
                  variant=variant)
    show_plot('numeric_vs_analytic.pdf')
    _, ax = pyplot.subplots(1, 1, figsize=(10, 5))
    plot_difference_arithmetic(settings=settings,
                  frame_index=-1,
                  ax=ax,
                  history=history,
                  arithmetic_by_mc=arithmetic_by_mc,
                  S_linspace=simulation.S,
                  option_type=f'arithmetic {variant}')
    show_plot('numeric_vs_analytic_error.pdf')
    
    print(f"True price: {params['true_price']}")
    print(f"Numeric price: {np.interp(params['spot'], simulation.S, history[-1][:,0])}")
    print(f"MC price: {np.interp(params['spot'], simulation.S, arithmetic_by_mc)}")

In [ ]:
params = {
        'T': 0.25,
        'K': 95,
        'r': 0.1,
        'sgma': 0.1,
        'spot': 100,
        'true_price': 6.132,
        'S_max': 120,
        'S_min': 80
    }
run_numeric_and_mc(params, nx=121, ny=500, nt=1500)